# CDC Mortality Data 1968-2018

In [81]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

The CDC Mortality Data is publically available here:
https://www.cdc.gov/nchs/data_access/cmf.htm
The mortality data from 1968-1988 is housed in two zip files. In order to conduct my analysis I had to download the zip files to my computer.  Due to the CDC's privacy concerns, I am not comfortable publishing the text data to github. The data can be deciphered with the codes provided by the CDC in a pdf found on the webpage given.  Mortality Data from 1989-2018 can be found by using their WONDER api found here:
https://wonder.cdc.gov/mortSQL.html and 
https://wonder.cdc.gov/ucd-icd10.html

## Importing and Cleaning Data

In [82]:
#This creates a dataframe for mortality data by US county from 1968-1978
file=r'Data\CDC_Wonder\mort6878\Mort6878.txt'
with open(file) as f:
    content=f.readlines()
content=[x.strip() for x in content]
FIPS=[]
for code in content:
    code=code[0:5]
    FIPS.append(code)
year_list=[]
for year in content:
    year=year[5:9]
    year_list.append(year)
deaths=[]
for death in content:
    death=death[19:]
    deaths.append(death)
dfMort6878=pd.DataFrame({'FIPS':FIPS,'year':year_list,'deaths':deaths})
dfMort6878['deaths']=dfMort6878['deaths'].str.lstrip()
dfMort6878.head(10)

,FIPS,year,deaths
0,01001,1968,1
1,01001,1968,2
2,01001,1968,1
3,01001,1968,2
4,01001,1968,1
5,01001,1968,1
6,01001,1968,1
7,01001,1968,1
8,01001,1968,1
9,01001,1968,1


In [83]:
len(dfMort6878.iloc[0,2])

1

In [84]:
dfMort6878.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8774864 entries, 0 to 8774863
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   FIPS    object
 1   year    object
 2   deaths  object
dtypes: object(3)
memory usage: 200.8+ MB


In [85]:
#this does the same thing but for 1979-1988
file=r'Data\CDC_Wonder\mort7988\Mort7988.txt'
with open(file) as f:
    content=f.readlines()
content=[x.strip() for x in content]
FIPS=[]
for code in content:
    code=code[0:5]
    FIPS.append(code)
year_list=[]
for year in content:
    year=year[5:9]
    year_list.append(year)
deaths=[]
for death in content:
    death=death[19:]
    deaths.append(death)
dfMort7988=pd.DataFrame({'FIPS':FIPS,'year':year_list,'deaths':deaths})
dfMort7988['deaths']=dfMort7988['deaths'].str.lstrip()
dfMort7988.head(10)

,FIPS,year,deaths
0,01001,1979,1
1,01001,1979,1
2,01001,1979,1
3,01001,1979,1
4,01001,1979,2
5,01001,1979,1
6,01001,1979,1
7,01001,1979,1
8,01001,1979,1
9,01001,1979,1


In [86]:
dfMort7988.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8776385 entries, 0 to 8776384
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   FIPS    object
 1   year    object
 2   deaths  object
dtypes: object(3)
memory usage: 200.9+ MB


In [87]:
#this groups the mortality data by year and FIPS code 
df=pd.concat([dfMort6878,dfMort7988])
df['year']=df.year.astype(int)
df['deaths']=df.deaths.astype(int)
df=df.groupby(['FIPS','year']).sum()
df6888=df
df6888.head(10)

deaths
FIPS  year        
01001 1968     226
      1969     191
      1970     204
      1971     229
      1972     206
      1973     257
      1974     221
      1975     196
      1976     234
      1977     224

In [88]:
#This imports each cdc mortality data set from 1989-2016.  Raw text data file was initially cleaned up by hand.
df_list=[]
for i in range(1989,2017):
    file=r'Data\CDC_Wonder\Compressed_Mortality\Compressed Mortality, '+str(i)+' edit.txt'
    df=pd.read_csv(file,delim_whitespace=True)
    df=df.loc[df['Deaths']!='Missing']
    df=df.loc[df['Deaths']!='Suppressed']
    df_list.append(df)
for i,j in list(zip([x for x in range(len(df_list))],[x for x in range(1989,2017)])):
    df_list[i]['year']=j

In [89]:
#does the same thing for 2017-2018
df_list_short=[]
for i in range(2017,2019):
    file=r'Data\CDC_Wonder\Multiple_Cause\Multiple Cause of Death, '+str(i)+' edit.txt'
    df=pd.read_csv(file,delim_whitespace=True)
    df=df.loc[df['Deaths']!='Missing']
    df=df.loc[df['Deaths']!='Suppressed']
    df_list_short.append(df)
for i,j in list(zip([x for x in range(len(df_list_short))],[x for x in range(2017,2019)])):
    df_list_short[i]['year']=j
df_list.append(df_list_short[0])
df_list.append(df_list_short[1])

In [90]:
#combines and cleans up data
df=pd.concat(df_list)
df=df.drop(['County','Population','CrudeRate','Rel'], axis=1)
df.columns=['FIPS', 'deaths', 'year']
df['deaths']=df['deaths'].astype(int)
df['FIPS']=df['FIPS'].astype(str)
df.reset_index().drop(['index'], axis=1)
change=list(df.FIPS.values)
fips=[]
for element in change:
    if len(element)==4:
        element='0'+element
        fips.append(element)
    else:
        fips.append(element)
df.FIPS=fips
df=df.groupby(['FIPS','year']).sum()
df8918=df
df8918.head(10)

deaths
FIPS  year        
01001 1989     259
      1990     304
      1991     283
      1992     310
      1993     309
      1994     354
      1995     298
      1996     345
      1997     328
      1998     372

In [91]:
df=pd.concat([df6888,df8918])
df.head(10)

deaths
FIPS  year        
01001 1968     226
      1969     191
      1970     204
      1971     229
      1972     206
      1973     257
      1974     221
      1975     196
      1976     234
      1977     224

In [92]:
fips_list=list(df.reset_index().FIPS.unique())

In [93]:
df_list=[]
for fips in fips_list:
    df_list.append(df.loc[[fips]])

Let us make sure that each FIPS code has sufficient degrees of freedom to estimate, and to see if there are any discrepencies worth noting.  

In [94]:
test=[1]
for i in range(1,len(df_list)):
    if set(df_list[0].reset_index().year.values) == set(df_list[i].reset_index().year.values):
        test.append(1)
    else:
        test.append(0)
change_list=[]
for i, j in enumerate(test,start=0):
    if j==0:
        change_list.append(i)
change_list

[67,
 158,
 228,
 241,
 243,
 254,
 270,
 271,
 302,
 516,
 533,
 537,
 893,
 957,
 1398,
 1573,
 1584,
 1586,
 1602,
 1607,
 1619,
 1624,
 1627,
 1628,
 1629,
 1662,
 1667,
 1670,
 1676,
 1681,
 1682,
 1683,
 1707,
 1710,
 1716,
 1722,
 1723,
 1731,
 1776,
 1963,
 1992,
 2001,
 2003,
 2005,
 2341,
 2362,
 2368,
 2389,
 2390,
 2397,
 2509,
 2543,
 2579,
 2610,
 2623,
 2624,
 2627,
 2643,
 2648,
 2672,
 2689,
 2708,
 2709,
 2714,
 2751,
 2763,
 2886,
 2887,
 2888,
 2889,
 2891,
 2892,
 2893,
 2894,
 2895,
 2896,
 2897,
 2898,
 2899,
 2900,
 2902,
 2903,
 2904,
 2905,
 2906,
 2907,
 2908,
 2911,
 2912,
 2913,
 2914,
 2915,
 2916,
 2917,
 2918,
 2919,
 2920,
 2923,
 2924,
 2925,
 3115,
 3116,
 3117,
 3118,
 3119,
 3120,
 3121,
 3122,
 3123,
 3124,
 3125,
 3126,
 3127,
 3128,
 3129,
 3130,
 3131,
 3132,
 3133,
 3134,
 3135,
 3136,
 3137,
 3138,
 3139,
 3140,
 3141,
 3142,
 3143,
 3144,
 3145,
 3146,
 3147,
 3148,
 3149,
 3150]

In [95]:
len(change_list)

136

In [96]:
len(df_list)

3151

For convenience we will get rid of these FIPS codes that have incomplete datasets 

In [97]:
for index in change_list[::-1]:
    del df_list[index]

## Modeling

### ARI Model

In [69]:
import warnings
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(df, arima_order):
    # prepare training dataset
    df['deaths']=df['deaths'].astype(float)
    X=df.deaths.values
    train, test = X[:(len(X)-2)], X[(len(X)-2):]
    # make predictions
    model = ARIMA(train, order=arima_order)
    model_fit = model.fit(disp=0)
    pred = model_fit.forecast(steps=len(test))
    # calculate out of sample error
    mse = mean_squared_error(test, pred[0])
    mae=mean_absolute_error(test, pred[0])
    return mse, mae
 
def arima_dict(p_values,d_values,q_values):
    emptydict={}
    for p in p_values:
        if p==0:
            for d in d_values:
                for q in q_values:
                    order=(p,d,q)
                    emptydict[str(order)]=[]
        else:
            for d in d_values:
                for q in [0]:
                    order=(p,d,q)
                    emptydict[str(order)]=[]
    return emptydict
                
def evaluate_models(df, p_values, d_values, q_values,dict_mse,dict_mae):
    temp_mse=[]
    temp_mae=[]
    for p in p_values:
        if p==0:
            for d in d_values:
                for q in q_values:
                    order = (p,d,q)
                    mse, mae=evaluate_arima_model(df,order)
                    temp_mse.append(mse)
                    temp_mae.append(mae)
        else:
            for d in d_values:
                for q in [0]:
                    order = (p,d,q)
                    mse, mae=evaluate_arima_model(df,order)
                    temp_mse.append(mse)
                    temp_mae.append(mae)
    temp_mse.sort()
    temp_mae.sort()
    dict_mse[str(order)].append(temp_mse[0])
    dict_mae[str(order)].append(temp_mae[0])
p_values = [0, 1, 2,3,4,5]
d_values = [1]
q_values = [0,1,2]
dict_mse=arima_dict(p_values,d_values,q_values)
dict_mae=arima_dict(p_values,d_values,q_values)
warnings.filterwarnings("ignore")
for i in range(len(df_list)):
    evaluate_models(df_list[i],p_values,d_values,q_values,dict_mse,dict_mae)

In [70]:
mse_dict={}
for order, mse_list in dict_mse.items():
    mse_dict[order]=np.sum(np.array(mse_list))
mae_dict={}
for order, mae_list in dict_mae.items():
    mae_dict[order]=np.sum(np.array(mae_list))

In [71]:
mse_dict

{'(0, 1, 0)': 0.0,
 '(0, 1, 1)': 0.0,
 '(0, 1, 2)': 0.0,
 '(1, 1, 0)': 0.0,
 '(2, 1, 0)': 0.0,
 '(3, 1, 0)': 0.0,
 '(4, 1, 0)': 0.0,
 '(5, 1, 0)': 9931074.658272207}

In [72]:
mae_dict

{'(0, 1, 0)': 0.0,
 '(0, 1, 1)': 0.0,
 '(0, 1, 2)': 0.0,
 '(1, 1, 0)': 0.0,
 '(2, 1, 0)': 0.0,
 '(3, 1, 0)': 0.0,
 '(4, 1, 0)': 0.0,
 '(5, 1, 0)': 77536.2240820878}

ARI(5,1) has the best performance for all FIPS code, and has a combined MAE of 77536.224

### Facebooks Prophet

def facebook_wonder(df_list):
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error 
    """This fuction takes a list of dataframes and runs facebooks prophet on each one, and provides a list of"""
    """MSE and MAE"""
    from fbprophet import Prophet
    MSE_List_FB=[]
    MAE_List_FB=[]
    for test in df_list:
        test=pd.DataFrame(test.reset_index()).drop(columns='FIPS').iloc[:-2]
        X=test.iloc[-2:].deaths.values
        test.columns=['ds','y']
        test['ds']=test['ds'].astype(str).apply(lambda x:x+('-12-31'))
        test['ds']=pd.to_datetime(test['ds'])
        model = Prophet()
        model.fit(test)
        future = ['2017-12-31','2018-12-31']
        future = pd.DataFrame(future)
        future.columns = ['ds']
        future['ds']= pd.to_datetime(future['ds'])
        forecast = model.predict(future)
        yhat=forecast.yhat.values
        MSE=mean_squared_error(X,yhat)
        MAE=mean_absolute_error(X,yhat)
        MSE_List_FB.append(MSE)
        MAE_List_FB.append(MAE)
    return MSE_List_FB, MAE_List_FB

MSE_List_FB, MAE_List_FB = facebook_wonder(df_list)

In [65]:
np.sum(np.array(MSE_List_FB))

48262254.82871482

In [66]:
np.sum(np.array(MAE_List_FB))

102755.14544662925

It appears the ARI(5,1,0) model performs the best.  Lets use it to make our predictions for 2020 deaths for each FIPS code.

In [108]:
def arima_model_predict(df, arima_order):
    # prepare training dataset
    df['deaths']=df['deaths'].astype(float)
    df=pd.DataFrame(df.reset_index())
    train=df.deaths.values
    fips=[df['FIPS'].values[0]]
    model = ARIMA(train, order=arima_order)
    model_fit = model.fit(disp=0)
    pred = model_fit.forecast(steps=2)
    death=[pred[0][1]]
    death_L95=[pred[2].reshape(1,-1)[0][2]]
    death_U95=[pred[2].reshape(1,-1)[0][3]]
    df_new=pd.DataFrame({'FIPS': fips, 'deaths':death, 'deaths_L95':death_L95,'deaths_U95':death_U95})
    return df_new
df_list_new=[]
for i in range(len(df_list)):
    df=arima_model_predict(df_list[i],(5,1,0))
    df_list_new.append(df)
df_final=pd.concat(df_list_new,axis=0)
df_final=df_final.set_index('FIPS')

In [109]:
df_final

,deaths,deaths_L95,deaths_U95
FIPS,,,
01001,545.776478,494.748833,596.804124
01003,2361.166953,2266.223528,2456.110379
01005,287.382131,235.903065,338.861196
01007,270.335151,242.329715,298.340587
01009,698.978032,642.538921,755.417143
...,...,...,...
56037,321.111036,274.586692,367.635379
56039,87.261790,70.713181,103.810398
56041,151.022869,130.413652,171.632086


Using the expected proportion of yearly deaths from February-October calculated in the 'CDC_Mortality_City' notebook, we can find the expected number of deaths in each FIPS code for this time period 

In [111]:
Oct_Prop=0.7385857521407692
Oct_Prop_L95=0.7283371932078238
Oct_Prop_U95=0.7473804733149562

In [115]:
df_final['OctTotal']=df_final['deaths']*Oct_Prop
df_final['OctTotal_L95']=df_final['deaths_L95']*Oct_Prop_L95
df_final['OctTotal_U95']=df_final['deaths_U95']*Oct_Prop_U95
df_final

,deaths,deaths_L95,deaths_U95,OctTotal,OctTotal_L95,OctTotal_U95
FIPS,,,,,,
01001,545.776478,494.748833,596.804124,403.102731,360.343976,446.039748
01003,2361.166953,2266.223528,2456.110379,1743.924270,1650.574883,1835.648937
01005,287.382131,235.903065,338.861196,212.256347,171.816976,253.258241
01007,270.335151,242.329715,298.340587,199.665691,176.497744,222.973929
01009,698.978032,642.538921,755.417143,516.255216,467.984995,564.584022
...,...,...,...,...,...,...
56037,321.111036,274.586692,367.635379,237.168036,199.991701,274.763504
56039,87.261790,70.713181,103.810398,64.450315,51.503040,77.585865
56041,151.022869,130.413652,171.632086,111.543339,94.985114,128.274469


Now lets get the Johns Hopkins University Covid-19 mortality numbers cleaned up.

# Johns Hopkins Cumulative Deaths Due to Covid-19

### Importing, Cleaning and Selecting Features 

We are looking for the cumulative reported deaths due to Covid-19 by FIPS code as of 07/31/20.  Johns Hopkins Covid-19 github repository can be found at https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

In [116]:
df_JHU=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
df_JHU.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,31,31,31,31,31,31,31,31,31,31
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,69,69,69,69,69,69,71,71,71,71
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,14,14,14,15,15,15,15,15,15,15
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,25,25,25,25,25,25,25,25,25,25


In [117]:
df_JHU=df_JHU[['FIPS','Admin2','Province_State','10/31/20']]
df_JHU.head()

,FIPS,Admin2,Province_State,10/31/20
0,1001.0,Autauga,Alabama,31
1,1003.0,Baldwin,Alabama,71
2,1005.0,Barbour,Alabama,9
3,1007.0,Bibb,Alabama,15
4,1009.0,Blount,Alabama,25


In [118]:
#Checking for null values
df_JHU[df_JHU['FIPS'].isnull()]

,FIPS,Admin2,Province_State,10/31/20
1267,NaN,Dukes and Nantucket,Massachusetts,2
1304,NaN,Federal Correctional Institution (FCI),Michigan,5
1336,NaN,Michigan Department of Corrections (MDOC),Michigan,75
1591,NaN,Kansas City,Missouri,213
2954,NaN,Bear River,Utah,18
2959,NaN,Central Utah,Utah,10
2978,NaN,Southeast Utah,Utah,5
2979,NaN,Southwest Utah,Utah,57
2982,NaN,TriCounty,Utah,3
2990,NaN,Weber-Morgan,Utah,36


In [119]:
df_JHU[df_JHU['FIPS'].isnull()]['10/31/20'].sum()

424

In [120]:
df_JHU=df_JHU[['FIPS','10/31/20']].dropna()
df_JHU.columns=['FIPS','deaths_10/31/20']
df_JHU['FIPS']=df_JHU['FIPS'].astype(int).astype(str).str.rstrip('.0').str.strip().apply(lambda x : x.zfill(5))
df_JHU.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3330 entries, 0 to 3339
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FIPS             3330 non-null   object
 1   deaths_10/31/20  3330 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.0+ KB


In [121]:
df_JHU=df_JHU.set_index('FIPS')
df_JHU

,deaths_10/31/20
FIPS,
01001,31
01003,71
01005,9
01007,15
01009,25
...,...
56039,1
56041,3
90056,10


Now lets merge the Johns Hopkins Data with the CDC death counts for 2020

In [122]:
df_final=df_final.join(df_JHU,how='inner')

In [132]:
df_final.head(20)

,deaths,deaths_L95,deaths_U95,OctTotal,OctTotal_L95,OctTotal_U95,deaths_10/31/20
FIPS,,,,,,,
01001,545.776478,494.748833,596.804124,403.102731,360.343976,446.039748,31
01003,2361.166953,2266.223528,2456.110379,1743.924270,1650.574883,1835.648937,71
01005,287.382131,235.903065,338.861196,212.256347,171.816976,253.258241,9
01007,270.335151,242.329715,298.340587,199.665691,176.497744,222.973929,15
01009,698.978032,642.538921,755.417143,516.255216,467.984995,564.584022,25
01011,118.220918,94.758469,141.683368,87.316286,69.016117,105.891383,17
01013,280.540579,241.946261,319.134898,207.203275,176.218460,238.515191,41
01015,1551.577264,1466.124325,1637.030204,1145.972861,1067.832876,1223.484408,65
01017,440.373917,393.663477,487.084357,325.253901,286.719752,364.037337,47
